In [2]:
import pandas as pd
import json
from pathlib import Path
# Try loading either 'jata.json' (user-specified) or fallback to 'data.json'
for fname in ('nag.json', 'nag.json'):
    p = Path(fname)
    if not p.exists():
        last_exc = FileNotFoundError(f"{fname} not found")
        continue
    with p.open('r', encoding='utf-8') as f:
        obj = json.load(f)
    # If JSON contains 'examallocation' use it, otherwise try to normalize the top-level list/dict
    if isinstance(obj, dict) and 'examallocation' in obj:
        df = pd.json_normalize(obj['examallocation'])
    elif isinstance(obj, list):
        df = pd.json_normalize(obj)
    else:
        try:
            df = pd.DataFrame(obj)
        except Exception as e:
            last_exc = e
            continue
    print(f"Loaded {fname} into DataFrame with shape {df.shape}")
    break
else:
    raise FileNotFoundError("Neither 'jata.json' nor 'data.json' could be read") from last_exc

# Display a quick preview
df.head()

Loaded nag.json into DataFrame with shape (849, 23)


,id,name,semester_id,course_id,course_section_id,max_mark,pass_mark,quiz_id,result_entry_layout,coursename,...,publish,quiz_type,status,score,remarks,outcomeresponse,assignment_status,assignment_upload,take_quiz,quiz_status
0,111289,Reflective Essay,542,5242,32288,200.00,120.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/200.00,-,0,Not Submitted,0,0,None
1,111279,Reflective Journal Writing 10,542,5242,32288,20.00,12.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/20.00,-,0,Not Submitted,0,0,None
2,111254,In Class Activities 6 : Group Discussion/ Thin...,542,5242,32288,20.00,12.00,NaN,TYPE1,Theory of Computation,...,0,None,Not Published,-/20.00,-,0,None,0,0,None
3,111277,Reflective Journal Writing 9,542,5242,32288,20.00,12.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/20.00,-,0,Not Submitted,0,0,None
4,111283,Assignment II,542,5242,32288,100.00,60.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/100.00,-,0,Not Submitted,0,0,None


In [3]:
try:
    print('Columns:')
    print(df.columns.tolist())
except NameError:
    import pandas as pd
    import json
    from pathlib import Path
    for fname in ('jata.json', 'data.json'):
        p = Path(fname)
        if not p.exists():
            last_exc = FileNotFoundError(f"{fname} not found")
            continue
        with p.open('r', encoding='utf-8') as f:
            obj = json.load(f)
        if isinstance(obj, dict) and 'examallocation' in obj:
            df = pd.json_normalize(obj['examallocation'])
        elif isinstance(obj, list):
            df = pd.json_normalize(obj)
        else:
            df = pd.DataFrame(obj)
        break
    print('Columns (loaded):')
    print(df.columns.tolist())


Columns:
['id', 'name', 'semester_id', 'course_id', 'course_section_id', 'max_mark', 'pass_mark', 'quiz_id', 'result_entry_layout', 'coursename', 'layoutname', 'start_date', 'end_date', 'publish', 'quiz_type', 'status', 'score', 'remarks', 'outcomeresponse', 'assignment_status', 'assignment_upload', 'take_quiz', 'quiz_status']


In [4]:
# Create term2 containing rows with the specified course_section_id values
term_ids = [32288, 32285, 32246, 32295]
term2 = df[df['course_section_id'].isin(term_ids)].copy()
print('term2 shape:', term2.shape)
term2.head()

term2 shape: (57, 23)


,id,name,semester_id,course_id,course_section_id,max_mark,pass_mark,quiz_id,result_entry_layout,coursename,...,publish,quiz_type,status,score,remarks,outcomeresponse,assignment_status,assignment_upload,take_quiz,quiz_status
0,111289,Reflective Essay,542,5242,32288,200.00,120.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/200.00,-,0,Not Submitted,0,0,None
1,111279,Reflective Journal Writing 10,542,5242,32288,20.00,12.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/20.00,-,0,Not Submitted,0,0,None
2,111254,In Class Activities 6 : Group Discussion/ Thin...,542,5242,32288,20.00,12.00,NaN,TYPE1,Theory of Computation,...,0,None,Not Published,-/20.00,-,0,None,0,0,None
3,111277,Reflective Journal Writing 9,542,5242,32288,20.00,12.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/20.00,-,0,Not Submitted,0,0,None
4,111283,Assignment II,542,5242,32288,100.00,60.00,NaN,TYPE9,Theory of Computation,...,0,None,Not Published,-/100.00,-,0,Not Submitted,0,0,None


In [5]:
# Print all unique values in the 'coursename' column
uniques = term2['coursename'].dropna().unique()

print('Unique coursename count:', len(uniques))
for v in sorted(uniques):
    print(v)

Unique coursename count: 4
Database Management Systems
Probability and Statistics
Research and Publication Ethics
Theory of Computation


In [6]:
# Remove rows where 'status' contains 'Not Published' (case-insensitive) and show remaining statuses
term2 = term2[~term2['status'].astype(str).str.contains('Not Published', case=False, na=False)].copy()
uniques = term2['status'].dropna().unique()
print('term2 shape after removal:', term2.shape)
print('Unique status count:', len(uniques))
for v in sorted(uniques):
    print(v)

term2 shape after removal: (35, 23)
Unique status count: 1
Published


In [7]:
term2.head()

,id,name,semester_id,course_id,course_section_id,max_mark,pass_mark,quiz_id,result_entry_layout,coursename,...,publish,quiz_type,status,score,remarks,outcomeresponse,assignment_status,assignment_upload,take_quiz,quiz_status
10,111087,In Class Activities 5 : Group Discussion/ Thin...,542,5029,32246,20.00,10.00,NaN,TYPE1,Probability and Statistics,...,1,None,Published,20.00/20.00,very good,0,None,0,0,None
11,111205,In Class Activities 6 : Group Discussion/ Thin...,542,5241,32285,20.00,10.00,NaN,TYPE1,Database Management Systems,...,1,None,Published,16.00/20.00,Good,0,None,0,0,None
17,111097,Reflective Journal Writing 7,542,5029,32246,20.00,10.00,NaN,TYPE9,Probability and Statistics,...,1,None,Published,16.00/20.00,give numerical example for each application,0,Submitted,0,0,None
21,111273,Reflective Journal Writing 7,542,5242,32288,20.00,12.00,NaN,TYPE9,Theory of Computation,...,1,None,Published,0.00/20.00,Assessment Not Submitted … Meet me in Person,0,Missed,0,0,None
22,111086,In Class Activities 4 : Group Discussion/ Thin...,542,5029,32246,20.00,10.00,NaN,TYPE1,Probability and Statistics,...,1,None,Published,13.00/20.00,discuss when your are paired,0,None,0,0,None


In [ ]:
# Aggregate total marks obtained and total possible marks per course from term2 (uses 'score' column)
# Parse 'score' values like '18.00/20.00' into obtained and max; fall back to 'max_mark' when needed
s = term2['score'].astype(str)
parts = s.str.split('/', n=1)
obtained = pd.to_numeric(parts.str[0].str.replace('[^0-9.]', '', regex=True), errors='coerce')
max_from_score = pd.to_numeric(parts.str[1].str.replace('[^0-9.]', '', regex=True), errors='coerce')
# If max not present in score string, use the 'max_mark' column where available
maxs = max_from_score.fillna(term2.get('max_mark'))
# Build a working copy with parsed numeric columns
agg_df = term2.assign(obtained=obtained, max_from_score=maxs)
# Group by coursename and sum obtained and max
summary = agg_df.groupby('coursename', dropna=False).agg(
    total_obtained=pd.NamedAgg(column='obtained', aggfunc='sum'),
    total_max=pd.NamedAgg(column='max_from_score', aggfunc='sum'),
    entries=pd.NamedAgg(column='obtained', aggfunc='count')
).reset_index()
# Compute percentage where possible
summary['percent'] = (summary['total_obtained'] / summary['total_max'] * 100).round(2)
# Sort by percent desc and display
summary = summary.sort_values(by='percent', ascending=False).reset_index(drop=True)
print('Totals per course (based on score column):')
print(summary.to_string(index=False))

Totals per course (based on score column):
                     coursename  total_obtained  total_max  entries  percent
Research and Publication Ethics           429.0      500.0        5    85.80
    Database Management Systems           134.0      230.0       10    58.26
          Theory of Computation            53.0      120.0        6    44.17
     Probability and Statistics           104.0      360.0       14    28.89


: 